In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

data = pd.read_csv('TrainLabels.csv')
data = data['ClipID']

for i in range(len(data)):
    data[i] = data[i][:-4]
data = data.to_frame()
#Matched pair label
data.insert(1,'frameid',data)
data1 = data.copy()
data1['frameid'] = np.random.permutation(data1['frameid'].values)
data.insert(2,'label',1)
data1.insert(2,'label',0)

data1 = data1.iloc[:,:].values

for i in range(len(data1)):
    if(data1[i,0]==data1[i,1]):
        data1[i,2] = 1
data1 = pd.DataFrame(data1)
data1.columns = ['ClipID','frameid','label']
data = data.append(data1)
data = shuffle(data)
data = data.reset_index()
data = data.drop(['index'],axis=1)
data.head(10)

,ClipID,frameid,label
0,4000181015,4000181015,1
1,4018350115,4018350115,1
2,4140810254,3100692035,0
3,3100642019,3100742055,0
4,1813740233,1813740233,1
5,3100741017,3100741017,1
6,2100571047,2100571047,1
7,4000181032,1100072083,0
8,1100172012,1100172012,1
9,3422270135,3422270135,1


In [2]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
import torch
from tqdm.autonotebook import tqdm
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset

/home/sanjaymoto75/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
data_path = './frames/train'

In [4]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
from tqdm.autonotebook import tqdm
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys
im_size = 224
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor()])
class video_dataset(Dataset):
    def __init__(self,frame_dir,train_csv,sequence_length = 60,transform = None):
        self.clip = train_csv['ClipID']
        self.frameid = train_csv['frameid']
        self.label = train_csv['label']
        self.frame_dir = frame_dir
        self.transform = transform
        self.sequence_length = sequence_length
        self.skip_length = int(300/sequence_length)
    def __len__(self):
        return len(self.frameid)
    def __getitem__(self,idx):
        id_1 = self.clip[idx][:6]
        path1 = os.path.join(self.frame_dir,id_1)
        path2 = os.path.join(path1,self.clip[idx])
        id_2 = self.frameid[idx][:6]
        path3 = os.path.join(self.frame_dir,id_2)
        path4 = os.path.join(path3,self.frameid[idx])
        seq_image = list()
        i = 0
        while i<300:
            path3 = os.path.join(path2,str(i)+'.jpg')
            image = cv2.imread(path3)
            if(self.transform):
                image = self.transform(image)
            seq_image.append(image)
            i = i+self.skip_length
        seq_image = torch.stack(seq_image)
        lb = self.label[idx]
        seq_image = seq_image.reshape(3,self.sequence_length,im_size,im_size)
        image = cv2.imread(os.path.join(path4,'10.jpg'))
        if(self.transform):
                image = self.transform(image)
        
        return seq_image,image,lb

In [5]:
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]

In [6]:
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.RandomRotation(degrees=10),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        #transforms.RandomRotation(degrees=10),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
train_data = video_dataset(data_path,data,transform = train_transforms)

In [7]:
train_loader = DataLoader(train_data,batch_size = 4,num_workers = 4 ,shuffle = True)
dataloaders = {'train':train_loader}

In [8]:
#Model t3d
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

__all__ = ['DenseNet', 'densenet121', 'densenet161'] # with DropOut

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv3d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv3d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv3d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))


class DenseNet3D(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000):

        super(DenseNet3D, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv3d(3, num_init_features, kernel_size=(3, 7, 7), stride=2, padding=(1, 3, 3), bias=False)),
            ('norm0', nn.BatchNorm3d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool3d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))
        # Linear layer
    def forward(self, x):
        features = self.features(x)
        return features
    

def densenet121_3D():
    model = DenseNet3D(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16))
    return model


def densenet121_3D_DropOut():
    model = DenseNet3D(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16), drop_rate=0.2)
    return model


def densenet169_3D():
    model = DenseNet3D(num_init_features=64, growth_rate=32, block_config=(6, 12, 32, 32))
    return model


# the below model has the lowest Top-1 error in ImageNet Data Set:
def densenet161_3D():
    model = DenseNet3D(num_init_features=96, growth_rate=48, block_config=(6, 12, 36, 24))
    return model


# the below model has the lowest Top-1 error in ImageNet Data Set:
def densenet161_3D_DropOut():
    model = DenseNet3D(num_init_features=96, growth_rate=48, block_config=(6, 12, 36, 24), drop_rate=0.2)
    return model


def densenet201_3D():
    model = DenseNet3D(num_init_features=64, growth_rate=32, block_config=(6, 12, 48, 32))
    return model


def densenet121(**kwargs):
    """Constructs a DenseNet-121_DropOut model.
    """
    model = DenseNet3D(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16), drop_rate=0.2, **kwargs)
    return model


def densenet161(**kwargs):
    """Constructs a DenseNet-161_DropOut model.
    """
    model = DenseNet3D(num_init_features=96, growth_rate=48, block_config=(6, 12, 36, 24), drop_rate=0.2, **kwargs)
    return model

In [9]:
#Model 2
from torchvision import models
class classifie(nn.Module):
    def __init__(self):
        super(classifie, self).__init__()
        self.cnn_arch = models.densenet121(pretrained = True)
        self.linear = nn.Linear(2048, 1024)
        self.model = self.cnn_arch.features
    def forward(self, x):
        features = self.model(x)
        out = F.relu(features, inplace=True)
        avg_pool = nn.functional.adaptive_avg_pool2d(out, output_size = 1)
        batch = out.shape[0]
        out = avg_pool.view(batch, -1)
        return out

In [15]:
t3d = densenet121_3D_DropOut().to('cuda')
t3d = nn.DataParallel(t3d)
linear = nn.Linear(2048,2).cuda()

In [16]:
classifier = classifie().to('cuda')
classifier = nn.DataParallel(classifier)

In [17]:
from tensorboardX import SummaryWriter
from Modular_code.clr import *
writer = SummaryWriter()
device = 'cuda'
cls_criterion = nn.CrossEntropyLoss().to(device)
params = list(t3d.parameters()) + list(classifier.parameters())
optimizer = torch.optim.Adam(params, lr=1e-3)
num_epochs = 5
onecyc = OneCycle(len(train_loader)*num_epochs,1e-3)

In [ ]:
from torch.autograd import Variable
iteration = 0
linear = nn.Linear(2048,2).cuda()
for epoch in range(1,num_epochs+1):
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    acc_all = list()
    loss_all = list()
    for phase in phase1:
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        for batch_i, (X,x1, y) in enumerate(dataloaders[phase]):
            iteration = iteration+1
            image_sequences = Variable(X.to(device), requires_grad=True)
            image = Variable(x1.to(device), requires_grad=True)
            labels = Variable(y.to(device), requires_grad=False)
            optimizer.zero_grad()
            #model.lstm.reset_hidden_state()
            features = t3d(image_sequences)
            out = F.relu(features, inplace=True)
            avg_pool = nn.functional.adaptive_avg_pool3d(out, output_size = 1)
            batch = out.shape[0]
            o1 = avg_pool.view(batch, -1)
            o2 = classifier(image)
            inp = torch.cat((o1,o2),1)
            predictions = linear(inp)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            if(phase=='train'):
                lr,_ = onecyc.calc()
                update_lr(optimizer, lr)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done

            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            if(phase=='train'):
                writer.add_scalar('data/acc',np.mean(epoch_metrics["acc"]) ,iteration)
                writer.add_scalar('data/loss',np.mean(epoch_metrics["loss"]) , iteration)
                acc_all.append(np.mean(epoch_metrics["acc"]))
                loss_all.append(np.mean(epoch_metrics["loss"]))
            if(phase=='val'):
                writer.add_scalar('data/valacc',np.mean(epoch_metrics["acc"]) , iteration)
                writer.add_scalar('data/valloss',np.mean(epoch_metrics["loss"]) , iteration)
        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        if(phase=='train'):

            acc_all.append(np.mean(epoch_metrics["acc"]))
            loss_all.append(np.mean(epoch_metrics["loss"]))


--- Epoch 1 ---

--- Phase train ---
[Epoch 1/5] [Batch 2678/2679] [Loss: 0.750674 (0.701616), Acc: 0.00% (49.98%)]]]
train , acc: 49.98133631952221

--- Epoch 2 ---

--- Phase train ---
[Epoch 2/5] [Batch 1490/2679] [Loss: 0.682977 (0.698085), Acc: 75.00% (49.66%)]]

In [ ]:
torch.save(t3d.module.state_dict(),('./t3dpre5.h5'))